In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

csv_path = "/content/drive/MyDrive/MACROBATT/FINAL_SURVIVOR_DEATH/SVM/pretrained_zero_shot_predicted_actual.csv"
Corpus = pd.read_csv(csv_path ,delimiter=',',encoding='latin-1')
Corpus.head()

,Filename,Text,API KEY Prediction,Predicted,ACTUAL
0,28353604_df.csv,A 69-year-old man became aware of the onset of...,HIGH,survivor,survivor
1,28559815_df.csv,"A 34-year-old woman, 1 week postpartum, presen...",NEUTRAL,survivor,survivor
2,28538413_df.csv,A 63-year-old male patient without smoking or ...,MODERATE,death,survivor
3,28353588_df.csv,A 75-year-old man was referred to our hospital...,NEUTRAL,death,survivor
4,28353596_df.csv,A 47-year-old female patient presented progres...,MODERATE,survivor,survivor


In [5]:
Corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Filename            200 non-null    object
 1   Text                200 non-null    object
 2   API KEY Prediction  200 non-null    object
 3   Predicted           200 non-null    object
 4   ACTUAL              200 non-null    object
dtypes: object(5)
memory usage: 8.0+ KB


In [11]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import defaultdict

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

# 1. Removing Blank Spaces
Corpus['Text'].dropna(inplace=True)

# 2. Changing all text to lowercase (if 'Text' column contains strings)
Corpus['text_original'] = Corpus['Text']
Corpus['Text'] = [entry.lower() if isinstance(entry, str) else entry for entry in Corpus['Text']]

# 3. Tokenization - In this, each entry in the corpus will be broken into a set of words
Corpus['Text'] = [word_tokenize(entry) if isinstance(entry, str) else entry for entry in Corpus['Text']]

# 4. Remove Stop words, Non-Numeric and performing Word Stemming/Lemmatization.
# WordNetLemmatizer requires Pos tags to understand if the word is a noun, verb, adjective, etc.
# By default, it is set to Noun
tag_map = defaultdict(lambda: wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

# Rest of your code for Lemmatization and further processing

Corpus.head()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Filename,Text,API KEY Prediction,Predicted,ACTUAL,text_original
0,28353604_df.csv,"[a, 69-year-old, man, became, aware, of, the, ...",HIGH,survivor,survivor,"[a, 69-year-old, man, became, aware, of, the, ..."
1,28559815_df.csv,"[a, 34-year-old, woman, ,, 1, week, postpartum...",NEUTRAL,survivor,survivor,"[a, 34-year-old, woman, ,, 1, week, postpartum..."
2,28538413_df.csv,"[a, 63-year-old, male, patient, without, smoki...",MODERATE,death,survivor,"[a, 63-year-old, male, patient, without, smoki..."
3,28353588_df.csv,"[a, 75-year-old, man, was, referred, to, our, ...",NEUTRAL,death,survivor,"[a, 75-year-old, man, was, referred, to, our, ..."
4,28353596_df.csv,"[a, 47-year-old, female, patient, presented, p...",MODERATE,survivor,survivor,"[a, 47-year-old, female, patient, presented, p..."


In [12]:
for index,entry in enumerate(Corpus['Text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

In [13]:
Corpus.drop(['Text'], axis=1)
output_path = '/content/drive/MyDrive/MACROBATT/FINAL_SURVIVOR_DEATH/SVM/preprocessed_svm.csv'
Corpus.to_csv(output_path, index=False)

In [14]:
Corpus

,Filename,Text,API KEY Prediction,Predicted,ACTUAL,text_original,text_final
0,28353604_df.csv,"[a, 69-year-old, man, became, aware, of, the, ...",HIGH,survivor,survivor,"[a, 69-year-old, man, became, aware, of, the, ...","['man', 'become', 'aware', 'onset', 'exertiona..."
1,28559815_df.csv,"[a, 34-year-old, woman, ,, 1, week, postpartum...",NEUTRAL,survivor,survivor,"[a, 34-year-old, woman, ,, 1, week, postpartum...","['woman', 'week', 'postpartum', 'present', 'de..."
2,28538413_df.csv,"[a, 63-year-old, male, patient, without, smoki...",MODERATE,death,survivor,"[a, 63-year-old, male, patient, without, smoki...","['male', 'patient', 'without', 'smoking', 'dri..."
3,28353588_df.csv,"[a, 75-year-old, man, was, referred, to, our, ...",NEUTRAL,death,survivor,"[a, 75-year-old, man, was, referred, to, our, ...","['man', 'refer', 'hospital', 'evaluation', 'dy..."
4,28353596_df.csv,"[a, 47-year-old, female, patient, presented, p...",MODERATE,survivor,survivor,"[a, 47-year-old, female, patient, presented, p...","['female', 'patient', 'present', 'progressivel..."
...,...,...,...,...,...,...,...
195,18815636_df.csv,"[a, 28-year-old, woman, was, referred, with, a...",MODERATE,survivor,survivor,"[a, 28-year-old, woman, was, referred, with, a...","['woman', 'refer', 'history', 'continuous', 'm..."
196,18666334_df.csv,"[a, 19-year-old, man, presented, at, the, emer...",NEUTRAL,survivor,survivor,"[a, 19-year-old, man, presented, at, the, emer...","['man', 'present', 'emergency', 'department', ..."
197,19860006_df.csv,"[a, 72-year-old, woman, underwent, rf, ablatio...",NEUTRAL,survivor,survivor,"[a, 72-year-old, woman, underwent, rf, ablatio...","['woman', 'underwent', 'rf', 'ablation', 'trea..."
198,28321070_df.csv,"[an, 84-year-old, woman, presented, to, a, loc...",NEUTRAL,survivor,survivor,"[an, 84-year-old, woman, presented, to, a, loc...","['woman', 'present', 'local', 'clinic', 'dyspn..."


In [15]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['ACTUAL'],test_size=0.2)

In [16]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [17]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

print(Tfidf_vect.vocabulary_)

{'man': 2339, 'become': 379, 'aware': 339, 'onset': 2696, 'exertional': 1436, 'dyspnea': 1250, 'july': 2164, 'history': 1846, 'smoke': 3894, 'year': 4984, 'cigarette': 625, 'per': 2848, 'day': 907, 'parent': 2800, 'brother': 478, 'lung': 2301, 'cancer': 508, 'past': 2812, 'medical': 2392, 'unremarkable': 4702, 'occupational': 2669, 'exposure': 1455, 'due': 1232, 'admit': 77, 'people': 2846, 'hospital': 1862, 'shunde': 3825, 'district': 1170, 'echocardiography': 1268, 'compute': 731, 'tomography': 4471, 'pulmonary': 3238, 'angiography': 191, 'ctpa': 874, 'indicate': 1997, 'severe': 3795, 'pah': 2759, 'arterial': 262, 'pressure': 3105, 'pap': 2784, 'right': 3581, 'heart': 1778, 'enlarge': 1348, 'ventricle': 4806, 'diameter': 1064, 'show': 3821, 'also': 141, 'thrombus': 4415, 'upper': 4715, 'lobe': 2276, 'artery': 264, 'however': 1868, 'region': 3424, 'affect': 94, 'pe': 2832, 'consider': 772, 'small': 3892, 'induce': 2001, 'addition': 57, 'connective': 764, 'tissue': 4451, 'disease': 113

In [18]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  85.0


In [19]:
print(classification_report(Test_Y,predictions_SVM))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.85      1.00      0.92        34

    accuracy                           0.85        40
   macro avg       0.42      0.50      0.46        40
weighted avg       0.72      0.85      0.78        40



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
